In [23]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [24]:
#Adapter layer
def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data,delimiter = sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def return_objects(bucket, arg_date, src_format):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    objects = [obj.key for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= min_date]
    return objects

In [25]:
#Application layer

def extract(bucket, objects):
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in objects], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN','Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'),closing_price_eur=('closing_price','min'),minimum_price_eur=('MinPrice','min'),maximum_price_eur=('MaxPrice','max'),daily_traded_volume=('TradedVolume','sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'],inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

def etl_report1(src_bucket, trg_bucket, objects, columns, arg_date, trg_key, trg_format):
    df = extract(src_bucket, objects)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True

In [26]:
# main function entrypoint

def main():
    #parameters/confirgurations
    #later read config
    arg_date = '2021-10-12'
    src_format = '%Y-%m-%d'
    src_bucket = 'deutsche-boerse-xetra-pds'
    trg_bucket = 'xetra-868686'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report'
    trg_format = '.parquet'
    
    #Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)

    #run application
    objects = return_objects(bucket_src, arg_date, src_format)
    etl_report1(bucket_src, bucket_trg, objects, columns, arg_date, trg_key, trg_format)

    

In [27]:
#run

main()

#Get opening price per ISIN and day

## Get closing pirce per ISIN and day

##Aggregations

## Percent Change Prev Closing

## Read the uploaded file

In [28]:
trg_bucket = 'xetra-868686'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report20211013_143021.parquet
xetra_daily_report20211013_203606.parquet
xetra_daily_report20211020_120112.parquet


In [31]:
prq_obj = bucket_trg.Object(key='xetra_daily_report20211020_120112.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [32]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT00000FACC2,2021-10-12,9.12,9.17,9.12,9.20,777,-0.43
1,AT00000FACC2,2021-10-13,9.24,9.31,9.24,9.32,600,1.53
2,AT00000FACC2,2021-10-14,9.08,9.07,9.02,9.11,55,-2.58
3,AT00000FACC2,2021-10-15,9.12,9.12,9.01,9.12,471,0.55
4,AT00000FACC2,2021-10-18,9.19,9.10,9.10,9.19,118,-0.22
...,...,...,...,...,...,...,...,...
21492,XS2314660700,2021-10-14,16.44,16.45,16.44,16.75,20,1.35
21493,XS2314660700,2021-10-15,16.73,16.98,16.73,16.98,0,3.23
21494,XS2314660700,2021-10-18,17.27,17.08,17.08,17.35,0,0.58
21495,XS2314660700,2021-10-19,17.24,17.15,17.15,17.61,6,0.42
